## Read and record resistance values using a B&K digital multimeter
## Application:  using a cailbrated thermistor to record temperatures

In [39]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


### Function to convert resistance to temperature

In [40]:
#This function converts thermistor resistance to absolute temperature
#It assumes you have already found the Steinhart-Hart coefficients a, b, c 
#As written, it assumes a 3-point calibration from the published R-T table from Vishay, the manufacturer
#You need to determine your own x array which contains the three fit parameters.  See last cell, where
#you measure R at 3 widely spaced temperatures, then solve the simultaneous equations in order
#to get your own unique x array containing the a, b, c coefficients.

x = array([  1.14284098e-03,   2.31872789e-04,   9.66213033e-08])

def tK(x,r):
    inverset = x[0]+x[1]*log(r)+x[2]*(log(r))**3         #solve for reciprocal T                            
    temperature = 1/inverset  #and report back the absolute temperature
    return temperature

### Code to establish communications between the PC and the B&K DMM

In [41]:
%pylab  
#the usual magic command that imports scipy, numpy, and matplotlib
# Python routine to control the B&K 5491B digital multimeter via a virtual serial port on USB.
# This was designed for use in a project lab for PHY310 Fall 2016
# 
# PJHT 8/24/16
#This cell needs to be run whenever you have disconnected the USB cable, shutoff the B&K, or closed the port
#Its purpose is to establish communications with the digital multimeter

import serial  #pySerial folder needs to be put in the directory pythonXX\lib\site-packages\serial
#The standard installation lacks this serial package.  If this notebook reports back that serial package
#is not found, open an Anaconda command window, then type
#>>>pip install pyserial
#then check that it's there with 
#>>>pip list
#The Anaconda 3 installer should put it in the correct directory

#Cabling: connect the B&K via a usb cable.  Drivers may need to be installed from the B&K website
#Because the device registers as a virtual comm port, Windows will put it at some port #
#This notebook attempts sequentially to connect the first device on com3-20 (there are better ways to do this)
#In Windows, pull up the Device Manager and look at the Ports (COM & lpt) listing to see where
#the B&K 5891 device has been placed.  If you have lots
#of other usb devices, there's a chance that these numbers won't work.  Modify accordingly.

#If you want two B&K 5491B DMMs connected, you'll need to identify them by serial number, which is 
#returned as the last response to the *IDN? query

import numpy as np
from numpy import append #need this to gather the data on the fly
from time import sleep, localtime,strftime #some timekeeping functions
import time
from serial import SerialException

#this function is used to send a command to the B&K
def send_to_BK(command_str):  #
    #print("Command String: ",command_str)
    return_bytes = bytearray(b'')
    command_str = command_str+"\n"  #append a <LF>, required for B&K
# In order to indicate the end of a line, you'd normally type <ENTER>, which on a (Win)PC would output
# two characters, carriage return (CR)+ linefeed (LF).  In Linux it would be just LF.  The default message
# terminator for the B&K is simply LF, which is encoded as \n at the end of each message, so that's what
# we use here.
    err = ser_BK.write(command_str.encode())
    #   the .encode() method converts the string into a byte array, and is required to use ser.write() routine
    sleep(SLEEP_TIME) #typically this is 0.3 seconds, because the B&K is a bit slow
    return_bytes = ser_BK.read(DATA_LENGTH)
    sleep(SLEEP_TIME)
    return return_bytes

PAUSE=0.5  #this is OK at 0, at least for 9600 baud

BAUD_BK = 38400  #The default rate is 9600 baud
'''
    This is the approximate transmission rate in bits per second.  Typically it takes about 10 bits to transmit
    a single character (1 start bit + 8 bits for the character + 1 or 2 stop bits), so figure that 
    9600 baud will do about 960 characters per second.

'''
TIME_OUT = 0.5  # don't hang around forever looking for data, but give the B& K time to respond

#This timeout limits the data rate
SLEEP_TIME = 0.2 #another pause in case  commands are garbled
DATA_LENGTH = 999  #could put this at 9999 and leave it


from serial.tools import list_ports  
list_ports.comports() #this handy function allows you to see present COM connections

#We don't know which virtual comm port the B&K is using, so we'll just check several:
return_bytes = bytearray()
connected = 0
print("Setting up B & K on USB virtual comm port")
for COM_PORT_NUM in np.arange(3,20):  #look for ports 3-20
    try:
        COM_PORT = "COM{:d}".format(COM_PORT_NUM)
        print('Trying comm port: '+COM_PORT)
        # The B&K throws errors every 60 seconds unless you specify some inter_byte_timeout
        ser_BK = serial.Serial(COM_PORT,BAUD_BK,timeout=TIME_OUT, inter_byte_timeout = 0.10)   
        sleep(SLEEP_TIME)
        return_bytes = send_to_BK("*IDN?")      #who are you?
        sleep(SLEEP_TIME)
        print(return_bytes)
        return_string = str(return_bytes,'utf-8')  #convert the bytes into a string
        response_list = return_string.split(",")   #so you can split up its parts on commas
       
        print("Connecting to B&K DMM")
        print("Here's the truncated return string: ")
        print(return_bytes[:23])
        if (return_bytes[:23] == b'*IDN?\n5491B  Multimeter'):  #This is the response from the 5491B series        
            print("B&K is connected on port {:d}\n".format(COM_PORT_NUM))
            print("Version: ",response_list[1])
            print("S/N: ",response_list[2])
            connected = 1
       
            break  
    except SerialException:   
        print ("Port already open....closing...opening")
        ser_BK.close()
        
    if (connected==1):      #got it; quit looking
        print("Connection is complete")
        break    
       
print(ser_BK.name)  #print the name of the com port again
print("Flushing B&K serial port buffers")
sleep(SLEEP_TIME)
ser_BK.flushInput() #flush input buffer, discarding all its contents
sleep(SLEEP_TIME)
ser_BK.flushOutput()#flush output buffer, aborting current output 
sleep(SLEEP_TIME)
#sleep(2)
print("Resetting the B%K....")
return_bytes=send_to_BK("*rst")  #reset
sleep(4)  #give it time to complete
print("Response bytes:  ", return_bytes)
print("\n\nCommunications established\n\n")

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib
Setting up B & K on USB virtual comm port
Trying comm port: COM3
b''
Connecting to B&K DMM
Here's the truncated return string: 
b''
Trying comm port: COM4
b'*IDN?\n5491B  Multimeter,Ver1.4.14.06.18,124D17187\n'
Connecting to B&K DMM
Here's the truncated return string: 
b'*IDN?\n5491B  Multimeter'
B&K is connected on port 4

Version:  Ver1.4.14.06.18
S/N:  124D17187

COM4
Flushing B&K serial port buffers
Resetting the B%K....
Response bytes:   b'*rst\n'


Communications established




### Here's where you can change what you measure with the DMM.  It can be a voltmeter, an ammeter, or an ohmmeter depending on the "func xxx" line

In [51]:
return_bytes=send_to_BK("func res")  #select function resistance (no ':' because it is top level)
sleep(1)
return_bytes=send_to_BK("res:nplc 1") #medium speed
sleep(1)
return_bytes = send_to_BK("res:rang:auto on") #auto ranging is OK
sleep(1)
print("Response bytes:  ", return_bytes)
return_bytes=send_to_BK("FUNC?")
print("Response bytes:  ", return_bytes)
sleep(2)
print("\n\nReady to take data\n\n")

Response bytes:   b'res:rang:auto on\n'
Response bytes:   b'FUNC?\nres\n'


Ready to take data




### Here is the loop in which data are acquired from the B&K
 Resistance values are converted into temperature and stored as lists
 
 The time at which the data are acquired is stored in a list


In [54]:
#Here's the actual data taking 
#First set up some file names and header text to be written at the top of the file
filename = "Thermistor resistance Data Logging cold.txt"
headertext = 'Block thermistor resistance in Modern Lab'
open(filename, mode ='w') #and open that filename for 'w'riting
# and here is where you can change the total number of readings that you will record:
MAX_COUNT=60*60*24  #good for 24 hours if you read 1/sec
MAX_COUNT = 5  #just try it for 30 readings
#MAX_COUNT =5  #or just 5

# Set up and clear the data lists for resistance, temperature, and time
res_list=[]  #keep track of where we've been
temp_list=[] # and the signal
time_list=[] #and the time, which will store in numerical format; convert later
done_flag = 0
count = 0
return_bytes = bytearray(b'')

print("Scanning..")
bad_count = 0
start_time = time.time()
while (count< MAX_COUNT):  #just a loop over MAX_COUNTs 
    sleep(1) #this 1 second pause, plus the pauses within send_to_BK, determine the acquisition period
    BK_str = "FETC?"  #This is the command to fetch the next fresh data
    return_bytes = send_to_BK(BK_str)#read the response from the B&K.  It's a byte array  
    print("resistance return string: [" ,return_bytes,"]") #show the complete byte array
    return_string = str(return_bytes,'utf-8')  #convert the bytes into a string (handy for further manipulation)
    trunc_string = return_string.replace('FETC?','') #and remove the fetch command from the string
    print("Stripped return string: ",trunc_string) #print just to make sure [sometimes it is empty]
    try:   #this try:...except is a way to catch errors without killing the program
        resistance = float(trunc_string) #extract the resistance from the string & convert to float
        #This will raise an error (an exception) if the number is garbled or not present
    except:
        continue  #sometimes the B&K gets out of phase because the FETC? command comes up empty
                  #just try again, kicking you back to the top of the while loop without advancing the 
                  #count counter
    #Here we assume the conversion has gone OK:
    print("resistance: ", resistance, "ohms")
    #And now let's put a couple of lines in to tell us if we've got problems with the connections:
    if resistance < 2:
        print("Are the two leads shorted against each other?")
        print("The resistance of the thermistor should lie in the 500 ohm to 20 Kohm range")
    elif resistance > 1e6:  #in Python, elif is "else-if"
        print("Are you measuring the resistance of an open circuit?")
        print("Check your connections")
    #print("Temperature: ", tK(x,resistance))
    #print("Resistance: ", resistance,"Temperature: ", tK(x,resistance))  
    #print("{:18.7f}".format(time.time()))  #a float version of time
    
    print(count, strftime("%H:%M:%S",localtime()), "Temperature: {:9.3f}, R = {:9.4f},\
        Err = ".format(tK(x,resistance)-273.15,resistance), bad_count)
    #(That print statement is too long to put on one line, so use the \ to break it across two lines)
    res_list.append(resistance) #append it to the list
    temp_list.append(tK(x,resistance))
    #time_list.append(strftime("%H:%M:%S",localtime()))
    time_list.append(time.time())
    #savetxt(filename,strftime("%H:%M:%S",localtime()),end=" ")
    #savetxt(filename,tK(x,resistance),end=" ")
    #savetxt(filename,resistance,fmt="%9.4f\n\r")
    count+=1

    #print("Temperature in C: {:9.2f}".format(tK(x,resistance)-273.15)      ) 

outp = column_stack((time_list,temp_list,res_list))  #TO DO:  format to allow H:M:S rather than float for time

savetxt(filename, outp, delimiter="  ", header = headertext)
    #savetxt(filename, (time_list[i],temp_list[i],res_list[i]), fmt="%H%M%S"+"%9.3f"+"%9.4f")
#savetxt(filename,column_stack((temp_list,res_list)),delimiter="   ", header=headertext)
time_arr = array(time_list)
#just an alternative way to save the data:
temp_arr = array(temp_list)
res_arr = array(res_list)
tc = zeros(time_arr.size,dtype=[('var1',float),('var2',float), ('var3',float)])
tc['var1']=time_arr
tc['var2']=temp_arr
tc['var3']=res_arr
np.savetxt('test2.txt', tc, fmt="%f %9.4f  %9.4f")

#print("Closing B&K port")
#ser_BK.close()
#print(wave_list)

Scanning..
resistance return string: [ b'FETC?\n3.30033945e4\n' ]
Stripped return string:  
3.30033945e4

resistance:  33003.3945 ohms
0 14:45:21 Temperature:    -0.236, R = 33003.3945,        Err =  0
resistance return string: [ b'FETC?\n3.30042109e4\n' ]
Stripped return string:  
3.30042109e4

resistance:  33004.2109 ohms
1 14:45:23 Temperature:    -0.236, R = 33004.2109,        Err =  0
resistance return string: [ b'FETC?\n3.30045039e4\n' ]
Stripped return string:  
3.30045039e4

resistance:  33004.5039 ohms
2 14:45:25 Temperature:    -0.236, R = 33004.5039,        Err =  0
resistance return string: [ b'FETC?\n3.30051406e4\n' ]
Stripped return string:  
3.30051406e4

resistance:  33005.1406 ohms
3 14:45:26 Temperature:    -0.237, R = 33005.1406,        Err =  0
resistance return string: [ b'FETC?\n3.30069609e4\n' ]
Stripped return string:  
3.30069609e4

resistance:  33006.9609 ohms
4 14:45:28 Temperature:    -0.238, R = 33006.9609,        Err =  0


### You can print out the resistance values like so

In [47]:
res_list  #run this line to get a list of all the resistance values you've recorded

[25.3535289, 25.354227, 25.3505687, 25.3544158, 25.3570308]

### Or plot them as a list

In [19]:
figure()  #and here is a rudimentary plot of the temperatures, assuming that the conversion is correct
plot(res_list)
#plot(temp_list)
show()

### This last function is essential for calibrating your thermistor.  You'll need to measure R at 3 widely spaced temperatures, then solve the simultaneous equations in order to get your own unique x array containing the a, b, c coefficients.  Here we illustrate the procedure by looking at the datasheet:

In [55]:
#  Solve simultaneous equations at three widely spaced temperatures
#Here we assume we have values at 10 C, 51 C, and 90 C, and looked at the Vishay datasheet
#which says that the resistance at 10 C is 19897 ohms, at 51 C it is 3547.5 ohms, and at 90 C it is 911.59 ohms
a = np.array([[1,log(3.3*10**4),(log(3.3*10**4))**3], [1,log(1.199*10**4),(log(1.199*10**4))**3],[1, log(1.923*10**3),(log(1.923*10**3))**3]])
b = np.array([1/(273.2),1/(294.8),1/(351.4)])
x = np.linalg.solve(a, b)  #Solve these equations to deduce a, b, c, and store in 3-element array x
x  #use this x in the tK function above
# Just to see what this looks like, we can generate an R T plot to get an idea:
figure()
r = linspace (500,20000, 1000) #create an array of 1000 pts between 500 and 20K ohms
# By the way:  do not extrapolate!  If you try to use your calibrated thermistor outside its range of calibration
# you will get increasingly bad results
t = tK(x,r)

title("Approximate calibration curve for Vishay thermistor, with \n \
    a = {:7e}, b = {:7e}, c = {:7e}".format(x[0],x[1],x[2]))
plt.xlabel('Absolute Temperature')
plt.ylabel('Resistance $\Omega$')
plt.plot(t,r)